# 📌 Projeto Pós-Graduação - Datathon Fase 5

## Objetivo
Construir um pipeline de Machine Learning para prever a aprovação ou reprovação de candidatos em processos seletivos, utilizando as bases fornecidas (`applicants.json`, `prospects.json`, `vagas.json`).  
O modelo será treinado com candidatos já aprovados/reprovados e aplicado nos candidatos que ainda estão "em andamento" no processo.

---

## Bases de Dados
1. **Applicants**  
   - Informações do candidato: dados pessoais, formação, experiências, currículos (`cv_pt`, `cv_en`).
   - Colunas aninhadas que precisam ser normalizadas (`infos_basicas`, `informacoes_pessoais`, `informacoes_profissionais`, etc.).

2. **Vagas**  
   - Detalhes das vagas abertas: título, modalidade, área, requisitos.
   - Contém também a lista de `prospects` vinculados a cada vaga.

3. **Prospects**  
   - Relaciona candidatos e vagas.
   - Coluna chave: `situacao_candidato` (status do processo seletivo).

---

## Situação do Target
A coluna `situacao_candidato` possui múltiplos status (ex.: *Aprovado, Reprovado, Em avaliação, Entrevista, Proposta Aceita*).  

- ~30% dos registros estão em **Aprovado/Reprovado** → servem de base para treino.  
- ~70% estão em **Em andamento** → onde aplicaremos o modelo.  

---

## Estratégia do Target
- **Aprovados (1):** `"Aprovado"`, `"Proposta Aceita"`, `"Contratado pela Decision"`.  
- **Reprovados (0):** `"Reprovado"`, `"Recusado"`, `"Não Aprovado pelo RH/Cliente/Requisitante"`, `"Desistiu"`, `"Sem interesse nesta vaga"`.  
- **Em andamento (NaN):** todos os demais status (ex.: `"Em avaliação pelo RH"`, `"Entrevista Técnica"`, `"Documentação CLT"`).

---

## Pipeline do Projeto

### 1. Pré-processamento
- Ler os três JSONs (`applicants`, `vagas`, `prospects`).
- Explodir e normalizar colunas aninhadas (listas/dicionários).
- Garantir identificadores únicos (`id_candidato`, `id_vaga`).

### 2. Integração
- Unir as bases:
  - `applicants + prospects` (via `id_candidato`).
  - Depois juntar com `vagas` (via `id_vaga`).
- Gerar o `df_final`.

### 3. Definição do Target
- Mapear `situacao_candidato` → `target`.
- Separar:
  - `df_train` → candidatos com target definido (aprovados/reprovados).
  - `df_predict` → candidatos em andamento (sem target).

### 4. Feature Engineering
- Texto: `cv_pt` → transformar com **TF-IDF**.
- Categóricas: `nivel_academico`, `area_atuacao`, etc. → OneHotEncoder.
- Numéricas: idade, tempo experiência (se disponível).

### 5. Modelagem
- Treinar modelos base: **Logistic Regression**, **RandomForest**.
- Comparar métricas (Acurácia, F1, Recall).
- Escolher o melhor e salvar em `.pkl`.

### 6. Aplicação
- Rodar o modelo no `df_predict`.
- Gerar probabilidade de aprovação para cada candidato em andamento.

### 7. Entrega Final
- Organizar repositório:


### Vagas

In [138]:
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')

In [139]:

with open("vagas.json", "r", encoding="utf-8") as f:
    jobs_raw = json.load(f)

df_jobs = pd.DataFrame.from_dict(jobs_raw, orient="index")
df_jobs.index.name = "id_vaga"

def flatten_sections(row):
    base   = {f"inf_{k}": v for k, v in row.get("informacoes_basicas", {}).items()}
    perfil = {f"perfil_{k}": v for k, v in row.get("perfil_vaga", {}).items()}
    benef  = {f"benef_{k}": v for k, v in row.get("beneficios", {}).items()}
    return {**base, **perfil, **benef}

flat_data = df_jobs.apply(flatten_sections, axis=1)
vagas = pd.DataFrame(list(flat_data), index=df_jobs.index).reset_index()

# Selecionar só as colunas úteis
colunas_util = [
    "id_vaga",
    "inf_titulo_vaga",
    "inf_cliente",
    "inf_vaga_sap",
    "perfil_nivel_academico",
    "perfil_nivel profissional",
    "perfil_nivel_ingles",
    "perfil_nivel_espanhol",
    "perfil_competencia_tecnicas_e_comportamentais",
    "perfil_principais_atividades"
]

df_vagas = vagas[colunas_util]


In [140]:
df_vagas.head()

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades
0,5185,Operation Lead -,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Avançado,Fluente,Required Skills:\n• Prior experience in Cloud ...,Operations Lead\n\nRoles & Responsibilities:\n...
1,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...
2,5183,ANALISTA PL/JR C/ SQL,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Analista,Nenhum,Intermediário,Requisitos mandatórios:\n\no Conhecimentos Téc...,Descrição – Atividades:\n\no Monitoramento das...
3,5182,Technical Architect - 11894809,Nelson-Page,Não,Ensino Superior Completo,Analista,Básico,Básico,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...
4,5181,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Mann and Sons,Não,Ensino Superior Completo,Sênior,Intermediário,Nenhum,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...


## Prospect

In [141]:

# 1) Ler prospects.json
with open("prospects.json", "r", encoding="utf-8") as f:
    prospects_raw = json.load(f)

# 2) Criar DataFrame onde cada índice = id da vaga
df_prospects = pd.DataFrame.from_dict(prospects_raw, orient="index").reset_index()
df_prospects.rename(columns={"index": "id_vaga"}, inplace=True)

# 3) Explodir lista de candidatos (prospects) -> cada candidato vira uma linha
df_vagas_explodida = df_prospects.explode("prospects").reset_index(drop=True)

# 4) Normalizar o dicionário de cada candidato
df_prospects_expandido = pd.json_normalize(df_vagas_explodida["prospects"])

# 5) Concatenar: vaga + info do candidato
prospects = pd.concat(
    [df_vagas_explodida.drop(columns=["prospects"]).reset_index(drop=True),
     df_prospects_expandido.reset_index(drop=True)],
    axis=1
)

# 6) Padronizar colunas importantes
# Em muitos dumps o id do candidato aparece como "codigo"
if "codigo" in prospects.columns:
    prospects.rename(columns={"codigo": "id_candidato"}, inplace=True)

# Se status vier como "situacao" ou "status"
if "situacao" in prospects.columns:
    prospects.rename(columns={"situacao": "situacao_candidado"}, inplace=True)
elif "status" in prospects.columns:
    prospects.rename(columns={"status": "situacao_candidado"}, inplace=True)

# Garantir tipos numéricos
prospects["id_vaga"] = pd.to_numeric(prospects["id_vaga"], errors="coerce").astype("Int64")
prospects["id_candidato"] = pd.to_numeric(prospects["id_candidato"], errors="coerce").astype("Int64")

# 7) Manter só colunas úteis (ajuste se quiser mais info da vaga)
df_prospects = prospects[["id_vaga", "id_candidato", "nome","situacao_candidado", "data_candidatura", "recrutador", "titulo"]].dropna(subset=["id_candidato"])


In [142]:
df_prospects["situacao_candidado"].unique()

array(['Encaminhado ao Requisitante', 'Contratado pela Decision',
       'Desistiu', 'Documentação PJ', 'Não Aprovado pelo Cliente',
       'Prospect', 'Não Aprovado pelo RH', 'Aprovado',
       'Não Aprovado pelo Requisitante', 'Inscrito', 'Entrevista Técnica',
       'Em avaliação pelo RH', 'Contratado como Hunting',
       'Desistiu da Contratação', 'Entrevista com Cliente',
       'Documentação CLT', 'Recusado', 'Documentação Cooperado',
       'Sem interesse nesta vaga', 'Encaminhar Proposta',
       'Proposta Aceita'], dtype=object)

###  Etapa Importante — Definição do Target

Um ponto crucial do projeto é a definição da variável **target**.  
Ao analisar a base, identificamos que parte dos candidatos já possui status finais, como **“Aprovado”**, **“Contratado”** ou **“Reprovado/Recusado”**.  

Esses casos serão utilizados como **conjunto de treino**:  
- **1 → Aprovado/Contratado**  
- **0 → Reprovado/Recusado**

Com esse histórico, o modelo aprende os padrões e características que diferenciam candidatos aprovados dos reprovados.  
Assim, conseguimos **aplicar o modelo nos candidatos em andamento** (ex.: *“Em avaliação”, “Entrevista técnica”*), prevendo a probabilidade de aprovação de cada um.  

Dessa forma, os recrutadores passam a ter um **ranking de candidatos mais promissores**, o que permite investir tempo apenas nos perfis com maior chance de sucesso.


In [143]:
map_status = {
    # APROVADOS
    "Aprovado": 1,
    "Proposta Aceita": 1,
    "Contratado pela Decision": 1,
    
    # REPROVADOS
    "Reprovado": 0,
    "Recusado": 0,
    "Não Aprovado pelo RH": 0,
    "Não Aprovado pelo Cliente": 0,
    "Não Aprovado pelo Requisitante": 0,
    
    # desistência também pode ser considerado reprovação
    "Desistiu": 0,
    "Desistiu da Contratação": 0,
    "Sem interesse nesta vaga": 0
}

# Cria coluna target (1 = aprovado, 0 = reprovado, NaN = em andamento)
df_prospects["target"] = df_prospects["situacao_candidado"].map(map_status)

In [144]:
df_prospects["target"].value_counts(dropna=False)

target
NaN    41783
0.0     9008
1.0     2968
Name: count, dtype: int64

In [120]:
df_prospects.head()

,id_vaga,id_candidato,nome,situacao_candidado,data_candidatura,recrutador,titulo,target
0,4530,25632,José Vieira,Encaminhado ao Requisitante,25-03-2021,Ana Lívia Moreira,CONSULTOR CONTROL M,NaN
1,4530,25529,Srta. Isabela Cavalcante,Encaminhado ao Requisitante,22-03-2021,Ana Lívia Moreira,CONSULTOR CONTROL M,NaN
2,4531,25364,Sra. Yasmin Fernandes,Contratado pela Decision,17-03-2021,Juliana Cassiano,2021-2607395-PeopleSoft Application Engine-Dom...,1.0
3,4531,25360,Alexia Barbosa,Encaminhado ao Requisitante,17-03-2021,Juliana Cassiano,2021-2607395-PeopleSoft Application Engine-Dom...,NaN
5,4533,26338,Arthur Almeida,Contratado pela Decision,29-04-2021,Stella Vieira,2021-2605708-Microfocus Application Life Cycle...,1.0


In [145]:
df_prospects = df_prospects[["id_vaga", "titulo", "id_candidato", "nome", "data_candidatura", "recrutador", "situacao_candidado", "target"]]

In [122]:
df_prospects.isnull().sum()

id_vaga                   0
titulo                    0
id_candidato              0
nome                      0
data_candidatura          0
recrutador                0
situacao_candidado        0
target                41783
dtype: int64

### Candidatos

In [146]:
with open("applicants.json", "r", encoding="utf-8") as f:
    applicants_raw = json.load(f)

# 2) Normalizar para DataFrame
df_applicants = pd.DataFrame.from_dict(applicants_raw, orient="index")
df_applicants.reset_index(inplace=True)
df_applicants.rename(columns={"index": "id_candidato"}, inplace=True)

# 3) Função para expandir colunas de dicionário
def expand_dict_column(df, col_name):
    if col_name in df.columns:
        df_expanded = pd.json_normalize(df[col_name])
        df_expanded.index = df.index
        df_expanded.columns = [f"{col_name}.{c}" for c in df_expanded.columns]
        return pd.concat([df.drop(columns=[col_name]), df_expanded], axis=1)
    return df

# 4) Expandir colunas aninhadas (incluímos formacao e cargo_atual)
cols_dict = [
    "infos_basicas", 
    "informacoes_pessoais", 
    "informacoes_profissionais", 
    "formacao_e_idiomas", 
    "cargo_atual"
]

for col in cols_dict:
    df_applicants = expand_dict_column(df_applicants, col)

# 5) Selecionar colunas úteis (incluindo skills e experiências)
colunas_util = [
    "id_candidato",
    "infos_basicas.objetivo_profissional",
    "informacoes_profissionais.titulo_profissional",
    "informacoes_profissionais.area_atuacao",
    "informacoes_profissionais.conhecimentos_tecnicos",
    "informacoes_profissionais.qualificacoes",
    "informacoes_profissionais.certificacoes",
    "informacoes_profissionais.experiencias",
    "formacao_e_idiomas.nivel_academico",
    "formacao_e_idiomas.nivel_ingles",
    "formacao_e_idiomas.nivel_espanhol",
    "cargo_atual.cargo_atual",
    "informacoes_profissionais.nivel_profissional",
    "formacao_e_idiomas.outro_idioma",
    "formacao_e_idiomas.cursos"

]

applicants = df_applicants[colunas_util].copy()

# 6) Renomear colunas para nomes mais claros
applicants.rename(columns={
    "cv_pt": "cv_texto_pt",
    "infos_basicas.objetivo_profissional": "objetivo_profissional",
    "informacoes_profissionais.titulo_profissional": "titulo_profissional",
    "informacoes_profissionais.area_atuacao": "area_atuacao",
    "informacoes_profissionais.conhecimentos_tecnicos": "conhecimentos_tecnicos",
    "informacoes_profissionais.qualificacoes": "qualificacoes",
    "informacoes_profissionais.certificacoes": "certificacoes",
    "informacoes_profissionais.experiencias": "experiencias",
    "formacao_e_idiomas.nivel_academico": "nivel_academico",
    "formacao_e_idiomas.nivel_ingles": "nivel_ingles",
    "formacao_e_idiomas.nivel_espanhol": "nivel_espanhol",
    "cargo_atual.cargo_atual": "cargo_atual",
    "informacoes_profissionais.nivel_profissional": "nivel_profissional",
    "formacao_e_idiomas.outro_idioma": "outro_idioma",
    "formacao_e_idiomas.cursos": "cursos"
}, inplace=True)

applicants = applicants.replace(r'^\s*$', pd.NA, regex=True)
applicants = applicants.replace(r'-', pd.NA, regex=True)
applicants = applicants.fillna("Não informado")

In [147]:
applicants.head()

,id_candidato,objetivo_profissional,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
0,31000,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
1,31001,Analista Administrativo,Analista Administrativo,Administrativa,Não informado,Não informado,Não informado,Não informado,Ensino Superior Incompleto,Nenhum,Nenhum,Não informado,Não informado,Não informado,Não informado
2,31002,Administrativo | Financeiro,Administrativo | Financeiro,Administrativa,Não informado,Não informado,Não informado,Não informado,Ensino Superior Completo,Intermediário,Básico,Não informado,Não informado,Não informado,Administração de Empresas
3,31003,Área administrativa,Área administrativa,Administrativa,Não informado,Não informado,Não informado,Não informado,Ensino Superior Incompleto,Nenhum,Nenhum,Não informado,Não informado,Não informado,Não informado
4,31004,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado


## Base Final Unificada

Após a leitura, tratamento e padronização das chaves (`id_vaga` e `id_candidato`), realizamos a junção dos três datasets:

- **Jobs (Vagas)** → informações sobre as vagas disponíveis.  
- **Prospects (Situação do candidato na vaga)** → status do processo seletivo (aprovado, recusado, em avaliação, etc).  
- **Applicants (Candidatos)** → dados do candidato: formação, experiência, idiomas, skills, etc.  

A junção foi feita em duas etapas:
1. **Jobs + Prospects** → unificação pelo campo `id_vaga`.  
2. **(Jobs + Prospects) + Applicants** → unificação final pelo campo `id_candidato`.  

### Resultado:
- **DataFrame final:** `df_full`  
- **Linhas:** representa cada candidato em uma vaga, com status e informações pessoais/profissionais.  
- **Colunas:** combinação de atributos da vaga, status do processo e características do candidato.  

Essa base final servirá como **entrada para a etapa de EDA e modelagem preditiva**, onde vamos definir variáveis explicativas (features) e a variável target (status aprovado/recusado).


In [148]:
df_vagas.head(1)

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades
0,5185,Operation Lead -,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Avançado,Fluente,Required Skills:\n• Prior experience in Cloud ...,Operations Lead\n\nRoles & Responsibilities:\n...


In [149]:
df_prospects.head(1)

,id_vaga,titulo,id_candidato,nome,data_candidatura,recrutador,situacao_candidado,target
0,4530,CONSULTOR CONTROL M,25632,José Vieira,25-03-2021,Ana Lívia Moreira,Encaminhado ao Requisitante,NaN


In [150]:

df_vagas["id_vaga"] = pd.to_numeric(df_vagas["id_vaga"], errors="coerce").astype("Int64")
df_prospects["id_vaga"] = pd.to_numeric(df_prospects["id_vaga"], errors="coerce").astype("Int64")


df_jobs_prospects = pd.merge(
    df_vagas,
    df_prospects,
    on="id_vaga",
    how="inner"
)

df_jobs_prospects["id_candidato"] = pd.to_numeric(df_jobs_prospects["id_candidato"], errors="coerce").astype("Int64")
applicants["id_candidato"] = pd.to_numeric(applicants["id_candidato"], errors="coerce").astype("Int64")

# Merge
df_full = pd.merge(
    df_jobs_prospects,
    applicants,
    on="id_candidato",
    how="inner"   # só quem existe nas duas bases
)


In [151]:
df_full.head(1)

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo,id_candidato,nome,data_candidatura,recrutador,situacao_candidado,target,objetivo_profissional,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
0,5185,Operation Lead -,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Avançado,Fluente,Required Skills:\n• Prior experience in Cloud ...,Operations Lead\n\nRoles & Responsibilities:\n...,Operation Lead -,11010,Dante Nascimento,11-05-2021,Srta. Bella Ferreira,Encaminhado ao Requisitante,NaN,CONSULTOR BASIS ECC/ S/4HANA / Solution Manage...,CONSULTOR BASIS ECC/ S/4HANA / Solution Manage...,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado


In [152]:
df_full.shape

(45071, 31)

In [153]:
df_full.head(1)

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo,id_candidato,nome,data_candidatura,recrutador,situacao_candidado,target,objetivo_profissional,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
0,5185,Operation Lead -,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Avançado,Fluente,Required Skills:\n• Prior experience in Cloud ...,Operations Lead\n\nRoles & Responsibilities:\n...,Operation Lead -,11010,Dante Nascimento,11-05-2021,Srta. Bella Ferreira,Encaminhado ao Requisitante,NaN,CONSULTOR BASIS ECC/ S/4HANA / Solution Manage...,CONSULTOR BASIS ECC/ S/4HANA / Solution Manage...,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado


In [155]:
df_full.to_csv("base_completa.csv", index=False)